In [624]:
from newsapi import NewsApiClient
import pandas as pd

# free newsapi only one month back 
newsapi = NewsApiClient(api_key="966334482cc84e8181e4a06fd9654a48")
#qintitle='Germany   AND (crash OR bitcoin)'    allows only 3 words for search,want to search for something like "closes" and "closed" use close*  
articles = list()
# 
# We are greedy, take every news we can get until we reach the limit
CountryList = ['Spanien','Frankreich','Italien','Deutschland','Österreich','Dänemark']
#Object = ["Grenzen","Schulen",Ausgangsbeschränkung,"Ausgangssperre","Einreisebschränkung","Notstand"]
ResponseList = ['Schlie*']
Object = ['Schulen']
CountryDic = {}
for land in CountryList:
    qInTitelVar = Object[0]+' AND '+ ResponseList[0]+' AND ' + land   # Parameter
    print(qInTitelVar)
    articles = []
    for n in range(1,100):
        try:
            headlines = newsapi.get_everything( qintitle = qInTitelVar,
            language='de',                                         from_param='2020-03-01',page=n)
            articles = articles + headlines['articles']
        except: # We reached the page limit of our account
            break
    print(len(articles))
    CountryDic[land] = {"ArticleList":articles,"Parameters": qInTitelVar}

Schulen AND Schlie* AND Spanien
0
Schulen AND Schlie* AND Frankreich
13
Schulen AND Schlie* AND Italien
35
Schulen AND Schlie* AND Deutschland
17
Schulen AND Schlie* AND Österreich
3
Schulen AND Schlie* AND Dänemark
3


In [625]:
def LikelyDayOfGovResponse(ArticleVar):
    df = pd.DataFrame(ArticleVar)
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    test = df.resample('D', on='publishedAt').count()
    mask = (df['publishedAt'] >= test['source'].argmax()) & (df['publishedAt'] < test['source'].argmax() + timedelta(days = 1))
    pd.set_option('display.max_colwidth', 20000)
    df['source'] = df['source'].apply(lambda r : r['name'])
    dfArgmax = df[mask].drop(columns=['author','urlToImage','content'])
    return(dfArgmax)

In [595]:
def LikelyDayOfGovResponse(ArticleVar):
    df = pd.DataFrame(ArticleVar)
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    test = df.resample('D', on='publishedAt').count()
    mask = (df['publishedAt'] >= test['source'].argmax()) & (df['publishedAt'] < test['source'].argmax() + timedelta(days = 1))
    pd.set_option('display.max_colwidth', 20000)
    df['source'] = df['source'].apply(lambda r : r['name'])
    dfArgmax = df[mask].drop(columns=['author','urlToImage','content'])
    return(dfArgmax)

In [626]:
artVar = CountryDic.get('Frankreich').get('ArticleList')
ResponseDf = LikelyDayOfGovResponse(artVar)
ResponseDf["MatchParameter"] = CountryDic.get('Frankreich').get("Parameters")
ResponseDf

C:\Users\eddy-\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


,description,publishedAt,source,title,url,MatchParameter
0,"In Frankreich bleiben ab Montag alle Schulen, Universitäten und Kindergärten geschlossen. Präsident Macron sagt, das Land stehe erst am Anfang einer Pandemie.",2020-03-12 20:44:34,Faz.net,"Frankreich lässt Schulen, Unis, Kindergärten schließen",https://www.faz.net/aktuell/politik/ausland/frankreich-laesst-schulen-unis-kindergaerten-schliessen-16677096.html,Schulen AND Schlie* AND Frankreich
1,Der französische Staatspräsident Emmanuel Macron hat drastische Maßnahmen zur Eindämmung des Coronavirus verordnet. Alle Schulen und Kitas in Frankreich bleiben geschlossen. Außerdem rief er alle Franzosen zu weiteren Handlungen auf.,2020-03-12 19:57:45,Welt.de,"Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.welt.de/vermischtes/article206526735/Coronavirus-Frankreich-schliesst-alle-Schulen-und-Kitas.html,Schulen AND Schlie* AND Frankreich
2,None,2020-03-12 19:38:10,Orf.at,Frankreich schließt bis auf Weiteres alle Schulen,https://orf.at/stories/3157628/,Schulen AND Schlie* AND Frankreich
3,Merkel: Wegen Coronavirus auf Sozialkontakte weitgehend verzichten +++ Bund und Länder: Ab Montag alle planbaren Operationen verschieben +++ Dow Jones erleidet schwersten Verlust seit 33 Jahren +++ Die wichtigsten überregionalen Entwicklungen im Newsblog.,2020-03-12 19:22:00,Berliner-zeitung.de,"Coronavirus: Frankreich schließt alle Schulen, Hochschulen und Kitas - Berliner Zeitung",https://www.berliner-zeitung.de/gesundheit-oekologie/zahl-der-corona-toten-in-deutschland-steigt-auf-sechs-li.5643,Schulen AND Schlie* AND Frankreich
4,"Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen t-online.de Macron kündigt massive Hilfen an:Frankreich schließt alle Schulen n-tv NACHRICHTEN ""Mehr zum Thema"" ansehen in Google News",2020-03-12 20:26:00,T-online.de,Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen - t-online.de,https://www.t-online.de/nachrichten/panorama/id_87512202/coronavirus-kampf-gegen-pandemie-frankreich-schliesst-alle-schulen.html,Schulen AND Schlie* AND Frankreich
5,"Im Kampf gegen die Coronavirus-Krise hat Frankreichs Präsident Emmanuel Macron die Schließung aller Schulen, Hochschulen und Kindertagesstätten im Land angekündigt.",2020-03-12 19:36:03,Stern.de,"Macron: Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.stern.de/news/macron--frankreich-schliesst-alle-schulen--hochschulen-und-kitas-ab-montag-9180158.html,Schulen AND Schlie* AND Frankreich
6,"Das Coronavirus breitet sich weiter aus, Veranstaltungen werden reihenweise abgesagt, das öffentliche Leben kommt mehr und mehr zum Erliegen. In Deutschland gibt es derzeit fünf Todesfälle und mehr als 1.500 bestätigte Corona-Infizierte. Weltweit sind es bere…",2020-03-12 19:23:40,Focus,"Corona-Krise - Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag - weiterer Todesfall in Heinsberg",https://www.focus.de/gesundheit/news/corona-krise-frankreich-schliesst-alle-schulen-hochschulen-und-kitas-ab-montag-weiterer-todesfall-in-heinsberg_id_11765515.html,Schulen AND Schlie* AND Frankreich
8,Europäische Zentralbank verkündet Hilfspaket + Halle schließt Schulen und Kitas + Coronavirus erreicht den Deutschen Bundestag + Dax stürzt ab + Der Newsblog.,2020-03-12 19:21:36,Der Tagesspiegel,"Frankreich schließt ab Montag alle Schulen, Unis und Kitas",https://www.tagesspiegel.de/wissen/kampf-gegen-das-coronavirus-frankreich-schliesst-ab-montag-alle-schulen-unis-und-kitas/25560996.html,Schulen AND Schlie* AND Frankreich
9,Mit drastischen Worten bereitet Präsident Macron die Franzosen auf Einschnitte im öffentlichen Leben vor. Infolge des Corona-Virus befinde sich das Land in der schlimmsten Gesundheitskrise seit einem Jahrhundert. Bis auf weiteres schließen landesweit alle Bil…,2020-03-12 19:58:43,N-tv.de,Macron kündigt massive Hilfen an: Frankreich schließt alle Schulen,https://www.n-tv.de/panorama/Frankreich-schliesst-alle-Schulen-article21

In [612]:
ResponseDf

,description,publishedAt,source,title,url,MatchParameter
0,"In Frankreich bleiben ab Montag alle Schulen, Universitäten und Kindergärten geschlossen. Präsident Macron sagt, das Land stehe erst am Anfang einer Pandemie.",2020-03-12 20:44:34,Faz.net,"Frankreich lässt Schulen, Unis, Kindergärten schließen",https://www.faz.net/aktuell/politik/ausland/frankreich-laesst-schulen-unis-kindergaerten-schliessen-16677096.html,Schulen AND Schlie* Dänemark
1,Der französische Staatspräsident Emmanuel Macron hat drastische Maßnahmen zur Eindämmung des Coronavirus verordnet. Alle Schulen und Kitas in Frankreich bleiben geschlossen. Außerdem rief er alle Franzosen zu weiteren Handlungen auf.,2020-03-12 19:57:45,Welt.de,"Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.welt.de/vermischtes/article206526735/Coronavirus-Frankreich-schliesst-alle-Schulen-und-Kitas.html,Schulen AND Schlie* Dänemark
2,None,2020-03-12 19:38:10,Orf.at,Frankreich schließt bis auf Weiteres alle Schulen,https://orf.at/stories/3157628/,Schulen AND Schlie* Dänemark
3,Merkel: Wegen Coronavirus auf Sozialkontakte weitgehend verzichten +++ Bund und Länder: Ab Montag alle planbaren Operationen verschieben +++ Dow Jones erleidet schwersten Verlust seit 33 Jahren +++ Die wichtigsten überregionalen Entwicklungen im Newsblog.,2020-03-12 19:22:00,Berliner-zeitung.de,"Coronavirus: Frankreich schließt alle Schulen, Hochschulen und Kitas - Berliner Zeitung",https://www.berliner-zeitung.de/gesundheit-oekologie/zahl-der-corona-toten-in-deutschland-steigt-auf-sechs-li.5643,Schulen AND Schlie* Dänemark
4,"Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen t-online.de Macron kündigt massive Hilfen an:Frankreich schließt alle Schulen n-tv NACHRICHTEN ""Mehr zum Thema"" ansehen in Google News",2020-03-12 20:26:00,T-online.de,Coronavirus: Kampf gegen Pandemie – Frankreich schließt alle Schulen - t-online.de,https://www.t-online.de/nachrichten/panorama/id_87512202/coronavirus-kampf-gegen-pandemie-frankreich-schliesst-alle-schulen.html,Schulen AND Schlie* Dänemark
5,"Im Kampf gegen die Coronavirus-Krise hat Frankreichs Präsident Emmanuel Macron die Schließung aller Schulen, Hochschulen und Kindertagesstätten im Land angekündigt.",2020-03-12 19:36:03,Stern.de,"Macron: Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag",https://www.stern.de/news/macron--frankreich-schliesst-alle-schulen--hochschulen-und-kitas-ab-montag-9180158.html,Schulen AND Schlie* Dänemark
6,"Das Coronavirus breitet sich weiter aus, Veranstaltungen werden reihenweise abgesagt, das öffentliche Leben kommt mehr und mehr zum Erliegen. In Deutschland gibt es derzeit fünf Todesfälle und mehr als 1.500 bestätigte Corona-Infizierte. Weltweit sind es bere…",2020-03-12 19:23:40,Focus,"Corona-Krise - Frankreich schließt alle Schulen, Hochschulen und Kitas ab Montag - weiterer Todesfall in Heinsberg",https://www.focus.de/gesundheit/news/corona-krise-frankreich-schliesst-alle-schulen-hochschulen-und-kitas-ab-montag-weiterer-todesfall-in-heinsberg_id_11765515.html,Schulen AND Schlie* Dänemark
8,Europäische Zentralbank verkündet Hilfspaket + Halle schließt Schulen und Kitas + Coronavirus erreicht den Deutschen Bundestag + Dax stürzt ab + Der Newsblog.,2020-03-12 19:21:36,Der Tagesspiegel,"Frankreich schließt ab Montag alle Schulen, Unis und Kitas",https://www.tagesspiegel.de/wissen/kampf-gegen-das-coronavirus-frankreich-schliesst-ab-montag-alle-schulen-unis-und-kitas/25560996.html,Schulen AND Schlie* Dänemark
9,Mit drastischen Worten bereitet Präsident Macron die Franzosen auf Einschnitte im öffentlichen Leben vor. Infolge des Corona-Virus befinde sich das Land in der schlimmsten Gesundheitskrise seit einem Jahrhundert. Bis auf weiteres schließen landesweit alle Bil…,2020-03-12 19:58:43,N-tv.de,Macron kündigt massive Hilfen an: Frankreich schließt alle Schulen,https://www.n-tv.de/panorama/Frankreich-schliesst-alle-Schulen-article21637864.html,Schulen AND Schlie* Dänemark
10,"Weg